In [1]:
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [2]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [3]:
for i in range(len(y_train_valid)):
    if(y_train_valid[i] == 769):
        y_train_valid[i] = 0
    if(y_train_valid[i] == 770):
        y_train_valid[i] = 1
    if(y_train_valid[i] == 771):
        y_train_valid[i] = 2
    if(y_train_valid[i] == 772):
        y_train_valid[i] = 3

In [4]:
for i in range(len(y_test)):
    if(y_test[i] == 769):
        y_test[i] = 0
    if(y_test[i] == 770):
        y_test[i] = 1
    if(y_test[i] == 771):
        y_test[i] = 2
    if(y_test[i] == 772):
        y_test[i] = 3

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
datatype = torch.float
torch.cuda.device(device)
print(device)

cuda:0


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, (1, 5)),
            nn.BatchNorm2d(16)) 
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 16, (22, 1)),
            nn.ELU(),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(1, 3))
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 32, (1, 5)),
            nn.ELU(),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(1, 3))
        self.conv4 = nn.Sequential(
            nn.Conv2d(32, 64, (1, 5)),
            nn.ELU(),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(1, 3))   
        self.conv5 = nn.Sequential(
            nn.Conv2d(64, 128, (1, 5)),
            nn.ELU(),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(1, 3)) 
        self.conv6 = nn.Sequential(
            nn.Conv2d(128, 256, (1, 5)),
            nn.ELU(),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(1, 3))
        self.fc1 = nn.Sequential(
            nn.Linear(256*3, 128),
            nn.ELU(),
            nn.Dropout())
        self.fc2 = nn.Sequential(
            nn.Linear(128, 64),
            nn.ELU(),
            nn.Dropout()) 
        self.fc3 = nn.Sequential(
            nn.Linear(64, 20),
            nn.ELU(),
            nn.Dropout()) 
        self.fc4 = nn.Sequential(
            nn.Linear(20, 4),
            nn.ELU(),
            nn.Dropout()) 
        self.softmax = nn.Softmax(1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)        
        x = self.softmax(x)
        return x

In [8]:
cnn = CNN().to(device)

In [10]:
lr = 1e-3
#lr_decay = 0.6
weight_decay = 1e-2
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=lr, weight_decay=weight_decay)

In [11]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4)

In [ ]:
batch_size = 300
num_iters = 1000
num_epoch = 2500
for epoch in range(num_epoch):
    batch_losses = []
    batch_accuracies = []
    for train_index, valid_index in kf.split(X_train_valid):
        X_train_cur = torch.tensor(X_train_valid[train_index].reshape(X_train_valid[train_index].shape[0], -1, 22, 1000), dtype=torch.float).to(device)
        X_valid_cur = torch.tensor(X_train_valid[valid_index].reshape(X_train_valid[valid_index].shape[0], -1, 22, 1000), dtype=torch.float).to(device)
        y_train_cur = torch.tensor(y_train_valid[train_index], dtype=torch.long).to(device)
        y_valid_cur = torch.tensor(y_train_valid[valid_index], dtype=torch.long).to(device)
        for it in np.arange(num_iters):
            X_batch = None
            y_batch = None
            num_total = 0
            index = np.random.choice(X_train_cur.shape[0], batch_size)
            X_batch = X_train_cur[index, :, :, :]
            y_batch = y_train_cur[index]
            optimizer.zero_grad()
            outputs = cnn(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            batch_losses.append(loss.item())
            predictions = torch.argmax(outputs, axis=1)
            accuracy = predictions.eq(y_batch).sum().item()
            num_total += y_batch.numel()
            batch_accuracies.append(accuracy/num_total)
        epoch_loss = sum(batch_losses) * 1.0 / len(batch_losses)
        epoch_accuracies = sum(batch_accuracies) * 1.0 / len(batch_accuracies)
        print("epoch_loss ------------",epoch_loss)
        print(epoch_accuracies)

epoch_loss ------------ 1.3861589349508285
0.25442666666666675
epoch_loss ------------ 1.3861545501947403
0.25431333333333317
epoch_loss ------------ 1.3861637066602708
0.2541711111111102
epoch_loss ------------ 1.3861719084084034
0.25423583333333194
epoch_loss ------------ 1.3861047860383988
0.2564399999999997


In [12]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()

In [14]:
input

tensor([[-1.0461,  0.2344,  2.2854,  1.1900, -0.5248],
        [-0.7498, -1.6367, -0.7026,  0.7583,  0.3610],
        [-0.8665,  1.6873, -0.1423, -0.2629,  1.3782]], requires_grad=True)